# Data analysis for contact angle measurements
In this notebook, the measured contact angle data is processed and analyzed.
The ntoebook contains 
- the accuracy measurement of the polynomial fit for interface approximation (Appendix 1)
- the contact angle line search study for Novec (Section 3.1.2, Appendix 2)
- the contact angle plot (Section 3.1.1)

In [ ]:
import pandas as pd
import os
from os.path import join
import matplotlib.pyplot as plt
from ast import literal_eval
import numpy as np
import matplotlib.patches as mpatches

## Functions

**User Input** : specify paths in function *get_paths_and_dataframes*

In [ ]:
def get_paths_and_dataframes(fluid, fit_degree):
    """
    Retrieves the paths and loads required dataframes required for further processing.
    This function enables simple result analysis of multiple fluids within one code block
    Paths are stores as global variables to avoid lengthy output.

    Args:
    fluid (str): The fluid being processed.
    fit_degree (int): The degree of polynomial fit used for contact angle measurement.

    Returns:
    None
    """

    # initialize global variables
    global path_exp_data, path_df_channel_edges, path_postproc_data, path_save, df_channel_edges, df_postproc, df_measured_contactangles

    # account for different naming conventions
    # for novec cases, the fluid name contains "front" or "back"
    fluid_position = fluid
    if 'front' in fluid or "back" in fluid:
        fluid = ''.join(s+'_' for s in fluid.split('_')[:-1])[:-1]

    # paths
    path_exp_data = "../data_experiments"
    path_postproc_data = join(path_exp_data, "case_parameters.xlsx")
    path_df_channel_edges = join(path_exp_data, fluid, "02_channel_edges","df_channel_edges.csv")
    path_df_contact_angles = join(path_exp_data, fluid, "03_contact_angle_measurement", 
                                f"df_measured_contactangles_poly{fit_degree}_{fluid_position}.csv")
    path_save = "../plots"

    # load data
    df_postproc =  pd.read_excel(path_postproc_data, sheet_name=fluid, index_col="case", skiprows=1)
    df_channel_edges = pd.read_csv(path_df_channel_edges, index_col='case')
    df_measured_contactangles = pd.read_csv(path_df_contact_angles, index_col='case')

    # create plot directory
    os.makedirs(path_save, exist_ok=True)

**Test paths and display dataframe**

In [ ]:
fluids = ["01_water"]
for f in fluids:
    get_paths_and_dataframes(f, 3)
    display(df_measured_contactangles)

## Curve fit accuracy estimation: RMS of distance

**User input: selection of parameters** 

In [ ]:
# fluids used in plots
fluids = ['01_water', '02_tween','03_novec_front', '03_novec_back']

# polynomial degrees that will be compared
fit_degrees = [2,3,4,10]

# plot cosmetics
labels_fluids=["Water", "Tween", "Novec front","Novec back"]
colors=[['#B95B22', '#E69F74'], #water-air
        ['#2E506C', '#94B6D2'], #tween-air
        ['#006600', '#87B486'], #novec-air
        ['#87B486', '#CEE0CE']] #novec-air

**Create plot**

In [ ]:
# loop over fluids
for f in range(len(fluids)):
    fluid=fluids[f]
    color = colors[f][0]
        
    # for novec, sepearte into "top" and "bottom" locations, for other fluids ignore     
    locations = [fluid[8:]+"_top", fluid[8:]+"_bot"] if "novec" in fluid else [""]
    location_labels = ["top","bottom"] if "novec" in fluid else [""]
    
    # loop over locations ("top" and "bottom" for novec)
    for l in range(len(locations)):
        
        # create figure
        fig2, ax2 = plt.subplots(figsize=(2,2))
    
        # get location
        location = locations[l]
        location_label = location_labels[l]
        
        # initialize lists
        rms_all = []
        rms_range_max = []
        rms_range_min = []
        
        # loop over fit degrees
        for i in range(len(fit_degrees)):
            
            fit_degree = fit_degrees[i]
            
            # get data
            get_paths_and_dataframes(fluid,fit_degree)
            cases = list(df_measured_contactangles.index)
            rms_all_cases = [np.mean(literal_eval(df_measured_contactangles[f"list_rms_dist_to_fit{location}_ransac"][c])) for c in cases]
        
            # average over all cases
            rms_dist_avg = np.mean( rms_all_cases)

            #error bar plot
            rms_all.append(rms_dist_avg)
            rms_range_max.append(max(rms_all_cases)-rms_dist_avg)        
            rms_range_min.append(rms_dist_avg-min(rms_all_cases))       
            
        # plot    
        ax2.errorbar(fit_degrees, rms_all, yerr=[rms_range_min,rms_range_max], fmt="o", 
                     c=color, markersize=4, linewidth=1, capsize=2)
        
        # cosmetics
        ax2.set_xscale('log')
        ax2.set_xticks([2,3,4,10], labels = [2,3,4,10], minor=False)
        plt.minorticks_off()
        ax2.set_xlabel("polynomial degree")
        ax2.set_ylabel('RMS distance (px)')
        ax2.set_title(f"{labels_fluids[f]} {location_label}", y=1.0, pad=-12, fontsize=10)
        ax2.set_ylim(0,5)
        
        # save plot
        fig2.savefig(join(path_save, f"rms_{fluid}{location}.png"), bbox_inches='tight', dpi=1000)

## Contact angle calibration study

**User input: selection of parameters** 

In [ ]:
# experimental contact angles that are used as reference for the given simulation cases.
# format:
#    case : [front contact angle, back contact angle]
theta_references_from_experiments = {
    '04_novec_sim_u1e-2_res30': [5,23], 
    '04_novec_sim_u1e-2_res100': [5,23],
    '04_novec_sim_u1e-1_res30': [34,50], 
    '04_novec_sim_u1e-1_res100': [34,50],
    '04_novec_sim_u1e-1_res30_avi0': [34,50],
    '04_novec_sim_u1e-1_res30_sl0': [34,50],
}

# contact contact angles used in the given simulation cases.
# format:
#    case : [tested contact angles in simulation]
theta_variation_simulations = {
    '04_novec_sim_u1e-2_res30':  [1,5,10,20,25,30],
    '04_novec_sim_u1e-2_res100':  [1,5,10,15,20,25,30],
    '04_novec_sim_u1e-1_res30': [30,35,40,45,50,55,60],
    '04_novec_sim_u1e-1_res100': [30,35,40,45,50,55,60],
    '04_novec_sim_u1e-1_res30_avi0': [30,35,40,45,50,55,60],
    '04_novec_sim_u1e-1_res30_sl0': [30,35,40,45,50,55,60],
}

# index of first result to be considered.
# Since the simulation shows interface oscillations in the first time steps
# after the initialization, the first results are ignored for the analysis.
index_start = 10

# polynomial degree of the used dataset.
fit_degree = 3

# plot cosmetics
colors=['darkblue','green','orange', 'k', 'g', 'lightblue', 'r']
s=[4,4,4]
linewidth=0.7

**Create plot**

In [ ]:
# loop over cases
for case in list(theta_variation_simulations.keys()):
    
    # get reference values
    thetas_sim = theta_variation_simulations[case]
    theta_references = theta_references_from_experiments[case]
    theta_ref_avg = np.mean(theta_references)
    
    # initialize list for results
    thetas_mean = [[],[]]

    # loop for front and back contact angle
    case_front_and_back = [case+'_front', case+'_back']
    for k in [0,1]:

        # set parameters
        fluid = case_front_and_back[k]
        theta_ref = theta_references[k]

        # get data
        get_paths_and_dataframes(fluid,fit_degree)
        subcases = list(df_measured_contactangles.index)   

        # create difference lists 
        for subcase in subcases:
            
            # get value
            theta = literal_eval(df_measured_contactangles["list_ca_ransac_bot"][subcase])

            # average theta over time
            theta_mean = np.mean(theta[index_start:])
            thetas_mean[k].append(theta_mean)

    # plot 
    fig1, ax1 = plt.subplots(figsize=(2.5,3))
    ax1.plot(thetas_sim, [theta_references[1]]*len(thetas_sim), '-',color=colors[1] , linewidth=linewidth,  label = r'$\theta_{exp, back} $')
    ax1.plot(thetas_sim, [theta_references[0]]*len(thetas_sim), '-',color=colors[0], linewidth=linewidth, label = r'$\theta_{exp, front} $')
    ax1.plot(thetas_sim, [np.mean(theta_references)]*len(thetas_sim), '-',color=colors[2] , markersize=s[0],  linewidth=linewidth, label = r'$\theta_{exp, mean} $')
    ax1.plot(thetas_sim, (thetas_mean[1]),'s--',color=colors[1],  linewidth=linewidth,  markersize=s[2], label =  r'$\theta_{sim, back}$')
    ax1.plot(thetas_sim, (thetas_mean[0]),'o--',color=colors[0], linewidth=linewidth, markersize=s[1], label =  r'$\theta_{sim, front}$')
    ax1.plot(thetas_sim, np.mean(thetas_mean,axis=0),'^--',color=colors[2], linewidth=linewidth,  label =  r'$\theta_{sim, mean}$')

    # cosmetics
    ax1.set_xlabel('prescribed contact angle (°)')
    ax1.set_ylabel(r'measured contact angle (°)')
    #ax1.set_ylim(-3,35)
    #ax1.set_ylim(10,65)
    ax1.set_title(case)
    ax1.legend(bbox_to_anchor=(1,1))
    
    # save figure
    fig1.savefig(join(path_save, f"ca_values_diff_{case}_deg{fit_degree}.png"), dpi=600, bbox_inches='tight')


## Contact angle plot

**User input: selection of parameters** 

In [ ]:
# select fluids for plot
fluids = ['01_water', '02_tween','03_novec_front', '03_novec_back']

# polynomial degree of the used dataset.
fit_degree = 3

# Exclude selected datasets from contact angle plot.
# A small part of the experimental results are not usable for contact angle measurements due to
# unstable interface movement or undetectable interface. These cases are used for regime detection,
# but not for contact angle measurements. It is ensured that at least 3 measurements are available for
# each given fluid and velocity.
exclude_datasets = {
    "01_water" : [],
    "02_tween" : [1,2, 10,11], 
    "03_novec_front" : [],
    "03_novec_back" : [],
}

# fluid properties
viscosities = {
    "01_water" : 0.9982e-3,
    "02_tween" : 0.9982e-3,
    "03_novec_front" : 1.2428e-3,
    "03_novec_back" : 1.2428e-3,
}
surfaceTensions = {
    "01_water" : 0.07274,
    "02_tween" : 0.035,
    "03_novec_front": 0.0162,
    "03_novec_back": 0.0162,
}

# plot cosmetics
colors=[['#B95B22', '#E69F74'], #water-air
        ['#2E506C', '#94B6D2'], #tween-air
        ['#006600', '#87B486'], #novec-air
        ['#87B486', '#CEE0CE']] #novec-air

**Create plot**

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))

for m in range(len(fluids)):
    
    fluid = fluids[m]
    print(f'\n {fluid}')
    
    # get data
    get_paths_and_dataframes(fluid,fit_degree)
    
    # drop cases
    df_postproc = df_postproc.drop([i for i in exclude_datasets[fluid] if i in df_postproc.index])
    df_measured_contactangles = df_measured_contactangles.drop([i for i in exclude_datasets[fluid] if i in df_measured_contactangles.index])

    cases = list(df_measured_contactangles.index)
    
    # ----- get array of velocities
    array_u = np.array(df_postproc['u'])
    array_cases = np.array(df_postproc['u'].index)
    set_u = sorted(list(set(array_u))) #unique set

    # ----- initialize
    data_top_all=[0 for k in range(len(set_u))]
    data_bot_all=[0 for k in range(len(set_u))]
    data_all=[0 for k in range(len(set_u))]

    # ------ loop over velocities to create boxplot
    for i in range(len(set_u)):
        u = set_u[i]
        print(f'\t velocity {np.round(u, 5)} m/s')
        
        # calc ca
        ca = np.array(set_u) * viscosities[fluid] / surfaceTensions[fluid]
        
        # get data for this velocity
        indices_u = np.where(array_u == u)
        number_cases = array_cases[indices_u]
        data_top = [literal_eval(df_measured_contactangles["list_ca_ransac_top"][c]) for c in number_cases]
        data_bot = [literal_eval(df_measured_contactangles["list_ca_ransac_bot"][c]) for c in number_cases]

        # collect all data
        data_top_all[i] = [j for i in data_top for j in i]
        data_bot_all[i] = [j for i in data_bot for j in i]
        data_all[i] = data_top_all[i] + data_bot_all[i]

        # median, mean and stdev
        median = np.median(data_all[i])
        upper_quartile = np.percentile(data_all[i], 75)
        lower_quartile = np.percentile(data_all[i], 25)
        mean = np.mean(data_all[i])
        std = np.std(data_all[i])  
        print(f"\t \t mean: {mean:.2f}° \t \t std: {std:.2f}°   = {std/mean*100:.2f} %" )
        print(f"\t \t median: {median:.2f}° \t quartile difference: {upper_quartile-lower_quartile:.2f}°" )

    # ----- Create boxplot
    medianprops = dict(linestyle='-', linewidth=2.5, color=colors[m][0])
    w = 0.2
    width = lambda p, w: 10**(np.log10(p)+w/2.)-10**(np.log10(p)-w/2.)
    bplot = ax.boxplot(data_all, positions=ca, widths=width(ca,w),
                        medianprops=medianprops, patch_artist=True,notch=True,
                        showfliers=True, whis=3)
        
    for box in bplot['boxes']:
        # change outline color
        box.set(color=colors[m][0], linewidth=1)
        # change fill color
        box.set(facecolor = colors[m][1] )
        
    for cap in bplot['caps']:
        # change outline color
        cap.set(color=colors[m][0], linewidth=1)
        
    for whisker in bplot['whiskers']:
        # change outline color
        whisker.set(color=colors[m][0], linewidth=1)
        
    for flier in bplot['fliers']:
        # change outline color
        flier.set(markeredgecolor=colors[m][0], linewidth=1)
        flier.set(marker='o', linewidth=1)

# cosmetics
ax.set_xscale('log')
ax.grid(True, linewidth=0.3)
ax.set_xlim(1e-6, 1e-2)
ax.set_xlabel("Capillary Number $Ca$")
ax.set_ylabel(r"Apparent contact angle $\theta$ ($\degree$)")

# save figure
plt.savefig(join(path_save, f"plot_ca_deg{fit_degree}.png"), dpi=1200, bbox_inches='tight')

In [ ]:
# create legend
labels = ['Water','Tween','Novec front','Novec back']

fig, ax = plt.subplots(figsize=(2,2))
for i in [0,1,2,3]:
    ax.plot([0,0], [0,0], label=labels[i], linewidth=4, color=colors[i][1])
    
ax.legend()

# save legend
#plt.savefig(join(path_save, "plot_ca_legend.png"), dpi=1200)